In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ta.trend import SMAIndicator, EMAIndicator, MACD, ADXIndicator
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.volatility import AverageTrueRange, BollingerBands
from ta.volume import ChaikinMoneyFlowIndicator

import sys
sys.path.append('../')

import config

# desactivate warnings
pd.options.mode.chained_assignment = None

/Users/dabereabasse/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
# Download Apple stock data at 1 hour intervals
start_date = pd.to_datetime(config.start_date) - pd.DateOffset(days=365)
end_date = config.end_date
print(f'Downloading Apple stock data from {start_date} to {end_date}')

aapl_data = yf.download(tickers='AAPL', interval='1d', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [6]:
aapl_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,3.067143,3.251429,3.041429,3.241071,2.733997,746015200
2009-01-05,3.327500,3.435000,3.311071,3.377857,2.849382,1181608400
2009-01-06,3.426786,3.470357,3.299643,3.322143,2.802384,1289310400
2009-01-07,3.278929,3.303571,3.223571,3.250357,2.741830,753048800
2009-01-08,3.229643,3.326786,3.215714,3.310714,2.792743,673500800


In [7]:
# Indicateurs de Tendance
# normalized SMA
aapl_data['SMA_10'] = SMAIndicator(close=aapl_data['Close'], window=10).sma_indicator() / aapl_data['Close']
aapl_data['SMA_15'] = SMAIndicator(close=aapl_data['Close'], window=15).sma_indicator() / aapl_data['Close']
aapl_data['SMA_20'] = SMAIndicator(close=aapl_data['Close'], window=20).sma_indicator() / aapl_data['Close']
aapl_data['SMA_50'] = SMAIndicator(close=aapl_data['Close'], window=50).sma_indicator() / aapl_data['Close']
aapl_data['SMA_100'] = SMAIndicator(close=aapl_data['Close'], window=100).sma_indicator() / aapl_data['Close'] 
aapl_data['SMA_200'] = SMAIndicator(close=aapl_data['Close'], window=200).sma_indicator() / aapl_data['Close']

# normalized EMA
aapl_data['EMA_10'] = EMAIndicator(close=aapl_data['Close'], window=10).ema_indicator() / aapl_data['Close']
aapl_data['EMA_12'] = EMAIndicator(close=aapl_data['Close'], window=12).ema_indicator() / aapl_data['Close']
aapl_data['EMA_14'] = EMAIndicator(close=aapl_data['Close'], window=14).ema_indicator() / aapl_data['Close']
aapl_data['EMA_26'] = EMAIndicator(close=aapl_data['Close'], window=26).ema_indicator() / aapl_data['Close']
aapl_data['EMA_30'] = EMAIndicator(close=aapl_data['Close'], window=30).ema_indicator() / aapl_data['Close']
aapl_data['EMA_50'] = EMAIndicator(close=aapl_data['Close'], window=50).ema_indicator() / aapl_data['Close']
aapl_data['EMA_100'] = EMAIndicator(close=aapl_data['Close'], window=100).ema_indicator() / aapl_data['Close']

# ADX
## ADX 14
adx = ADXIndicator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=14)
aapl_data['ADX_14'] = adx.adx()
aapl_data['ADX_14_neg'] = adx.adx_neg()
aapl_data['ADX_14_pos'] = adx.adx_pos()
## ADX 20
adx = ADXIndicator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=20)
aapl_data['ADX_20'] = adx.adx()
aapl_data['ADX_20_neg'] = adx.adx_neg()
aapl_data['ADX_20_pos'] = adx.adx_pos()
## ADX 25
adx = ADXIndicator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=25)
aapl_data['ADX_25'] = adx.adx()
aapl_data['ADX_25_neg'] = adx.adx_neg()
aapl_data['ADX_25_pos'] = adx.adx_pos()
## ADX 30
adx = ADXIndicator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=30)
aapl_data['ADX_30'] = adx.adx()
aapl_data['ADX_30_neg'] = adx.adx_neg()
aapl_data['ADX_30_pos'] = adx.adx_pos()

# Indicateurs de Momentum
# RSI
aapl_data['RSI_7'] = RSIIndicator(close=aapl_data['Close'], window=7).rsi()
aapl_data['RSI_14'] = RSIIndicator(close=aapl_data['Close'], window=14).rsi()
aapl_data['RSI_21'] = RSIIndicator(close=aapl_data['Close'], window=21).rsi()

# Stochastic Oscillator
## Stochastic Oscillator 14
stoch = StochasticOscillator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=14, smooth_window=3)
aapl_data['Stoch_14'] = stoch.stoch()
aapl_data['Stoch_14_signal'] = stoch.stoch_signal()
## Stochastic Oscillator 21
stoch = StochasticOscillator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=21, smooth_window=3)
aapl_data['Stoch_21'] = stoch.stoch()
aapl_data['Stoch_21_signal'] = stoch.stoch_signal()
## Stochastic Oscillator 28
stoch = StochasticOscillator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=28, smooth_window=3)
aapl_data['Stoch_28'] = stoch.stoch()
aapl_data['Stoch_28_signal'] = stoch.stoch_signal()

# Indicateurs de Volatilité
# normalized ATR
## ATR 14
atr = AverageTrueRange(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=14)
aapl_data['ATR_14'] = atr.average_true_range() / aapl_data['Close']
## ATR 20
atr = AverageTrueRange(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=20)
aapl_data['ATR_20'] = atr.average_true_range() / aapl_data['Close']
## ATR 28
atr = AverageTrueRange(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], window=28)
aapl_data['ATR_28'] = atr.average_true_range() / aapl_data['Close']

# Indicateurs de Volume
# Chaikin Money Flow
## CMF 14
cmf = ChaikinMoneyFlowIndicator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], volume=aapl_data['Volume'], window=14)
aapl_data['CMF_14'] = cmf.chaikin_money_flow()
## CMF 20
cmf = ChaikinMoneyFlowIndicator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], volume=aapl_data['Volume'], window=20)
aapl_data['CMF_20'] = cmf.chaikin_money_flow()
## CMF 28
cmf = ChaikinMoneyFlowIndicator(high=aapl_data['High'], low=aapl_data['Low'], close=aapl_data['Close'], volume=aapl_data['Volume'], window=28)
aapl_data['CMF_28'] = cmf.chaikin_money_flow()

In [8]:
aapl_data = aapl_data.loc[aapl_data.index >= config.start_date]
aapl_data.head()

,Open,High,Low,Close,Adj Close,Volume,SMA_10,SMA_15,SMA_20,SMA_50,...,Stoch_21,Stoch_21_signal,Stoch_28,Stoch_28_signal,ATR_14,ATR_20,ATR_28,CMF_14,CMF_20,CMF_28
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.447413,493729600,0.963623,0.945426,0.934846,0.933008,...,98.102255,92.072873,98.102255,92.072873,0.019581,0.019875,0.019913,0.192531,0.006533,-0.084723
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.458559,601904800,0.970799,0.949924,0.938145,0.932254,...,95.503529,93.621141,95.503529,93.621141,0.018931,0.019395,0.019558,0.124335,0.034848,-0.117104
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.355827,552160000,0.992530,0.969699,0.958527,0.947989,...,82.831675,92.145820,82.831675,92.145820,0.019379,0.019784,0.019923,0.083341,0.049708,-0.151361
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.344076,477131200,0.999221,0.976690,0.965220,0.950514,...,81.368461,86.567888,81.382317,86.572507,0.019029,0.019531,0.019747,0.110852,0.078307,-0.137121
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.386253,447610800,0.997283,0.975570,0.962190,0.945615,...,85.719876,83.306671,86.584861,83.599618,0.018544,0.019125,0.019412,0.206804,0.061121,-0.104320


In [9]:
# add target
aapl_data['target'] = aapl_data['Close'].pct_change().shift(-1) * 100

# drop Open, High, Low, Close, Adj Close, Volume
aapl_data = aapl_data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

In [11]:
# index to column
aapl_data = aapl_data.reset_index()
aapl_data.head()

,Date,SMA_10,SMA_15,SMA_20,SMA_50,SMA_100,SMA_200,EMA_10,EMA_12,EMA_14,...,Stoch_21_signal,Stoch_28,Stoch_28_signal,ATR_14,ATR_20,ATR_28,CMF_14,CMF_20,CMF_28,target
0,2010-01-04,0.963623,0.945426,0.934846,0.933008,0.883653,0.755846,0.968436,0.963540,0.959512,...,92.072873,98.102255,92.072873,0.019581,0.019875,0.019913,0.192531,0.006533,-0.084723,0.172893
1,2010-01-05,0.970799,0.949924,0.938145,0.932254,0.884416,0.757172,0.972808,0.967742,0.963475,...,93.621141,95.503529,93.621141,0.018931,0.019395,0.019558,0.124335,0.034848,-0.117104,-1.590632
2,2010-01-06,0.992530,0.969699,0.958527,0.947989,0.900728,0.771859,0.990617,0.985940,0.981842,...,92.145820,82.831675,92.145820,0.019379,0.019784,0.019923,0.083341,0.049708,-0.151361,-0.184871
3,2010-01-07,0.999221,0.976690,0.965220,0.950514,0.904477,0.775760,0.993824,0.989648,0.985839,...,86.567888,81.382317,86.572507,0.019029,0.019531,0.019747,0.110852,0.078307,-0.137121,0.664833
4,2010-01-08,0.997283,0.975570,0.962190,0.945615,0.900975,0.773125,0.989577,0.985710,0.982084,...,83.306671,86.584861,83.599618,0.018544,0.019125,0.019412,0.206804,0.061121,-0.104320,-0.882147


In [12]:
aapl_data.to_csv('../data/AAPL/financial_data.csv', index=False)